In [1]:
# TFIDF
from sklearn.feature_extraction.text import TfidfVectorizer
import joblib

#Doc2Vec
from sklearn.model_selection import train_test_split
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize
from gensim.models.doc2vec import Doc2Vec
from nltk.tokenize.treebank import TreebankWordDetokenizer

# general purpose
import os
import numpy as np
import pandas as pd
import time
import pickle
from tqdm import tqdm
from collections import Counter

#sklearn classifiers
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

# custom classes
from scripts.OccupationPreprocessor import OccupationPreprocessor
from scripts.TextPreprocessor import TextPreprocessor
from scripts.Embedder import Embedder

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\augus\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [27]:
files = ['./Data/ATP_gold_standard.xlsx', './Data/TFIDF_ATP_test_data_without_8000_samples_seed_42.csv']
input_columns = ['CURRENT_JOB_TITLE', 'input']
code_columns = ['NOC code by PATH', 'code']

TEST_FILE = 0

In [28]:
train_df = pd.DataFrame(pd.read_csv('./Data/doc2vec_cleaned_train_data_with_8000_ATP_seed_42.csv'))

# take data
# TODO fix this so it doesn't need to be an if statement, simple type check, 'int' does not have function "strip"
if TEST_FILE == 0:
    df = OccupationPreprocessor.prepare_df(
        files[TEST_FILE],
        input_column=input_columns[TEST_FILE],
        code_column=code_columns[TEST_FILE],
        n_digits=4
    )
    
elif TEST_FILE == 1:
    df = pd.read_csv(files[TEST_FILE])
    

# copy clean
d2v_data = df.copy()
d2v_data['input'] = d2v_data['input'].apply(TextPreprocessor.preprocess_text)

# copy uncleaned
tfidf_data = df.copy()
tfidf_data['input'] = tfidf_data['input'].apply(str.lower)

Input unprocessed by default


# Check for Exact Matches on Cleaned Data

In [29]:
predictions = pd.DataFrame()
predictions['exact_match'] = d2v_data['input'].apply(Embedder.check_exact_match, args=(train_df,))
predictions['code'] = d2v_data['code']

## How many did we correctly find in exact matches?

In [30]:
predictions['exact_matches_TP'] = predictions.apply(lambda row: str(row['exact_match']) == str(row['code']), axis=1)

# Load the 3 Trained Models

In [31]:
doc2vec_model = Doc2Vec.load('trial_11.model')

with open('TFIDF_SVM_KNN_LR.pkl', 'rb') as f:
    tfidf_classifiers = pickle.load(f)

tfidf_model = joblib.load('vectorizer.joblib')

# TFIDF Embedding

In [32]:
tfidf_input_vector = tfidf_model.transform(tfidf_data['input'])

# Make predictions with classifiers

In [33]:
start_time = time.time()
for clf in tfidf_classifiers.keys():
    predictions['{}_pred'.format(str(clf))] = tfidf_classifiers[clf].predict(tfidf_input_vector)
print('Time elapsed to predict on {} samples: {} seconds'.format(tfidf_input_vector.shape[0], time.time()-start_time))

Time elapsed to predict on 500 samples: 17.067100286483765 seconds


In [39]:
predictions['Ens_pred'] = predictions.apply(Embedder.ensemble_predict, axis = 1, args = (['LR_pred', 'KNN_pred', 'SVM_pred'], 'SVM_pred',))

In [40]:
predictions['Hybrid_pred'] = predictions.apply(
    lambda row : row['Ens_pred'] if row['exact_match'] == -1 else row['exact_match'], axis = 1
)

In [41]:
from sklearn.metrics import accuracy_score, f1_score

print(
    'Hybrid accuracy {}'.format( 
        accuracy_score(
            predictions['Hybrid_pred'], 
            predictions['code']
        )
     )
 )

Hybrid accuracy 0.55


In [43]:
predictions.sample(10)

,exact_match,code,exact_matches_TP,SVM_pred,KNN_pred,LR_pred,Ens_pred,Hybrid_pred
346,2174,6221,False,4154,11,9619,9619,2174
302,-1,712,False,712,712,7205,712,712
58,-1,4164,False,3111,5244,3111,3111,3111
10,-1,6341,False,6341,6562,6562,6562,6562
287,-1,2263,False,2222,2222,2222,2222,2222
238,-1,3211,False,4154,11,9619,9619,9619
297,9232,9232,True,9232,9232,9232,9232,9232
80,3141,3141,True,3141,3141,3141,3141,3141
222,-1,5131,False,5131,821,821,821,821
19,-1,2171,False,1112,1111,2171,2171,2171


In [25]:
tfidf_data[tfidf_data.index == 3479]

,input,code
3479,part owner,213


In [26]:
train_df[train_df['input'] == 'part owner']

,input,code
30783,part owner,1311


In [15]:
# Adding doc2vec prediction weight and exceptions post-processing